# Imports 

In [195]:
import pandas as pd
import numpy as np


# getting data

In [196]:
df = pd.read_csv('../data/que-faire-a-paris-.csv', sep=';')
new_df = df.copy()
df.columns

Index(['ID', 'URL', 'Titre', 'Chapeau', 'Description', 'Date de début',
       'Date de fin', 'Occurrences', 'Description de la date',
       'URL de l'image', 'Texte alternatif de l'image', 'Crédit de l'image',
       'Mots clés', 'Nom du lieu', 'Adresse du lieu', 'Code postal', 'Ville',
       'Coordonnées géographiques', 'Accès PMR', 'Accès mal voyant',
       'Accès mal entendant', 'Transport', 'Url de contact',
       'Téléphone de contact', 'Email de contact', 'URL Facebook associée',
       'URL Twitter associée', 'Type de prix', 'Détail du prix',
       'Type d'accès', 'URL de réservation', 'URL de réservation - Texte',
       'Date de mise à jour', 'Image de couverture', 'address_url_text',
       'address_url', 'address_text'],
      dtype='object')

Formatter les dates

In [197]:
df["Code postal"]

0       93170
1       75011
2       75005
3       75020
4       75007
        ...  
2171    75018
2172    75018
2173    75001
2174      NaN
2175      NaN
Name: Code postal, Length: 2176, dtype: object

In [198]:
columns_with_date = ['Date de début', 'Date de fin']
for col in columns_with_date:
    new_df[col] = pd.to_datetime(df[col], utc=True).dt.tz_localize(None)

In [199]:
df["Type de prix"].map

<bound method Series.map of 0       gratuit
1       gratuit
2       gratuit
3       gratuit
4       gratuit
         ...   
2171     payant
2172     payant
2173     payant
2174        NaN
2175        NaN
Name: Type de prix, Length: 2176, dtype: object>

Formatter le Type d'entrée

In [217]:
# possible values ['gratuit', 'payant', nan, 'gratuit sous condition']
def gratuit_ou_pas(x):
    if type(x) == float: # nan
        return "nan_pay" #on ne sait pas
    if "gratuit" in x:
        return "gratuit" #gratuit
    else:
        return "payant" #payant
def type_dacces(x):
    if type(x) == float: # nan
        return "nan_access" #on ne sait pas
    if "obligatoire" in x:
        return "obligatoire" #opbiligatoire
    if "non" in x:
        return "non" #non, Aucune idée ce que cv dire
    else:
        return "conseillee" #conseillée


new_df["Type de prix"] = df["Type de prix"].map(gratuit_ou_pas)
new_df["Type d'accès"] = df["Type d'accès"].map(type_dacces)
    

In [218]:
series = pd.to_datetime( np.arange(pd.Timestamp("2021-01-01 10:00:00"), pd.Timestamp("2021-12-30 22:00:22"), pd.Timedelta('1 H')) , utc=True).tz_localize(None)
# define columns
counting_cols = ["nb_events", "gratuit", "payant", "nan_pay", "nan_access", "obligatoire", "non", "conseillee"]
arrondissement_cols = ["7","14","6","1","8","2"]
all_cols = counting_cols + arrondissement_cols
max_duration = pd.Timedelta('7 days')

series = pd.DataFrame(series).assign(**dict.fromkeys(all_cols, 0))
series = series.set_index(0)

In [219]:
data= {}
for date in series.index:
    data[str(date)] = dict.fromkeys(all_cols, 0)
new_data ={}
for _, row in new_df.iterrows():
    if pd.isnull(row[columns_with_date[1]]) or pd.isnull(row[columns_with_date[0]]):
        # print("**",row[columns_with_date[1]])
        continue
    if row[columns_with_date[1]] - row[columns_with_date[0]] >= max_duration or row[columns_with_date[1]] >= pd.Timestamp("2021-12-29 22:00:22"):
        # print("--------->",row[columns_with_date[1]])
        continue
    for date in pd.date_range(row[columns_with_date[0]], row[columns_with_date[1]] + pd.Timedelta('1 H'), freq='H'):
        index = str(date.round('H'))
        # print(date.round('H'))
        new_row = data[index]
        new_row["nb_events"] += 1
        
        if row["Type de prix"] == "nan_pay":
            new_row["nan_pay"] += 1
        elif row["Type de prix"] == "gratuit":
            new_row["gratuit"] += 1
        elif row["Type de prix"] == "payant":
            new_row["payant"] += 1   
            
        if row["Type d'accès"] == "nan_access":
            new_row["nan_access"] += 1
        elif row["Type d'accès"] == "obligatoire":
            new_row["obligatoire"] += 1
        elif row["Type d'accès"] == "non":
            new_row["non"] += 1
        elif row["Type d'accès"] == "conseillee":
            new_row["conseillee"] += 1
        
        if pd.isna(row["Code postal"]):
            continue
        if "75" in row["Code postal"]:
            if "007" in row["Code postal"]:
                new_row["7"] += 1
            elif "14" in row["Code postal"]:
                new_row["14"] += 1
            elif "006" in row["Code postal"]:
                new_row["6"] += 1
            elif "001" in row["Code postal"]:
                new_row["1"] += 1
            elif "008" in row["Code postal"]:
                new_row["8"] += 1
            elif "002" in row["Code postal"]:
                new_row["2"] += 1
            else:
                continue
        else :
            continue
        
        
        new_data[index] = new_row
        # print(new_row)
        # print(data[index])
data = new_data

# Save

In [220]:
final = pd.DataFrame(data).T
final.sort_index(inplace=True)
# final
final.to_csv('../data/events_counting.csv')

# tests

In [67]:
new_df.groupby(columns_with_date[0]).count()

,ID,URL,Titre,Chapeau,Description,Date de fin,Occurrences,Description de la date,URL de l'image,Texte alternatif de l'image,...,Type de prix,Détail du prix,Type d'accès,URL de réservation,URL de réservation - Texte,Date de mise à jour,Image de couverture,address_url_text,address_url,address_text
Date de début,,,,,,,,,,,,,,,,,,,,,
2021-01-01 10:00:47,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,0,1,1,0,0,0
2021-01-09 09:30:54,1,1,1,1,1,1,1,1,1,1,...,1,0,1,0,0,1,1,0,0,0
2021-01-14 15:00:39,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,0,0,0
2021-01-19 15:00:00,1,1,1,1,1,1,1,1,1,0,...,1,1,1,0,0,1,1,0,0,0
2021-01-28 14:00:13,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-03 22:00:22,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,0,1,1,0,0,0
2022-07-05 22:30:32,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,0,0,0
2022-07-12 20:30:53,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,0,0,0


In [17]:
df

,ID,URL,Titre,Chapeau,Description,Date de début,Date de fin,Occurrences,Description de la date,URL de l'image,...,Type de prix,Détail du prix,Type d'accès,URL de réservation,URL de réservation - Texte,Date de mise à jour,Image de couverture,address_url_text,address_url,address_text
0,14122,https://www.paris.fr/evenements/conference-en-...,"Conférence "" Dedans - Dehors, Prendre l'air av...",Le théâtre l'Echangeur accueille vendredi 10 d...,<p>Dans son ouvrage <em>La conspiration des en...,2021-12-10 19:00:00,2021-12-10 21:00:00,2021-12-10T19:00:00+01:00_2021-12-10T21:00:00+...,Le vendredi 10 décembre 2021<br />de 19h00 à 2...,https://cdn.paris.fr/qfapv4/2021/11/30/medium-...,...,gratuit,NaN,conseillée,https://billetterie-lechangeur.mapado.com/even...,NaN,2021-11-30T14:37:14+01:00,https://opendata.paris.fr/api/v2/catalog/datas...,NaN,NaN,NaN
1,14067,https://www.paris.fr/evenements/seance-de-dedi...,Séance de dédicaces d'anciennes et anciens élè...,NaN,<p>Dans le cadre de l'exposition réalisée par ...,2022-01-15 10:00:00,2022-01-15 13:00:00,2022-01-15T10:00:00+01:00_2022-01-15T13:00:00+...,Le samedi 15 janvier 2022<br />de 10h00 à 13h0...,https://cdn.paris.fr/qfapv4/2021/11/27/medium-...,...,gratuit,Dans la limite des places disponibles.,NaN,NaN,NaN,2021-11-30T17:22:27+01:00,https://opendata.paris.fr/api/v2/catalog/datas...,NaN,NaN,NaN
2,11441,https://www.paris.fr/evenements/soiree-contes-...,Soirée Contes,"Quand un crapaud, une petite fille, un moineau...",<p>A l’initiative de la bibliothèque l’Heure j...,2021-12-04 18:00:07,2021-12-04 19:00:07,2021-12-04T18:00:07+01:00_2021-12-04T19:00:07+...,Le samedi 04 décembre 2021<br />de 18h00 à 19h...,https://cdn.paris.fr/qfapv4/2021/12/01/medium-...,...,gratuit,NaN,obligatoire,https://bibliotheques.paris.fr/,NaN,2021-12-01T14:13:25+01:00,https://opendata.paris.fr/api/v2/catalog/datas...,NaN,NaN,NaN
3,13582,https://www.paris.fr/evenements/arts-dans-la-r...,Arts dans la rue - 2021-2022 avec la compagnie...,Venez participer à des ateliers artistiques (t...,<p>La compagnie PLATEFORME vous invite du 4 dé...,2022-01-08 10:30:00,2022-02-19 12:30:00,NaN,Du samedi 04 décembre 2021 au samedi 25 juin 2...,https://cdn.paris.fr/qfapv4/2021/12/01/medium-...,...,gratuit,parisculture20eme@gmail.com,obligatoire,NaN,NaN,2021-12-01T15:13:55+01:00,https://opendata.paris.fr/api/v2/catalog/datas...,NaN,NaN,NaN
4,14182,https://www.paris.fr/evenements/conference-bas...,"Conférence - Baselitz, la rétrospective",NaN,<p><strong>Mercredi 26 janvier à 19h</strong><...,2022-01-26 19:00:00,2022-01-26 20:30:00,2022-01-26T19:00:00+01:00_2022-01-26T20:30:00+...,Le mercredi 26 janvier 2022<br />de 19h00 à 20...,https://cdn.paris.fr/qfapv4/2021/12/01/medium-...,...,gratuit,Ouverture des inscriptions le mardi 21/12/2021,obligatoire,NaN,NaN,2021-12-01T15:23:27+01:00,https://opendata.paris.fr/api/v2/catalog/datas...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2171,13316,https://www.paris.fr/evenements/amandine-beyer...,Amandine Beyer / Gli Incogniti en trio / Quatu...,Bijoux baroques\nSAM. 9/04\n\nBijoux classique...,<p>Bijoux baroques</p><p>SAM. 9/04</p><p><stro...,2022-04-09 17:00:27,2022-04-10 13:20:27,2022-04-09T16:00:27+01:00_2022-04-09T17:15:27+...,Le dimanche 10 avril 2022<br />de 11h00 à 12h2...,https://cdn.paris.fr/qfapv4/2021/11/08/medium-...,...,payant,5 € à 20 € par concert,obligatoire,https://www.theatredelaville-paris.com/fr/spec...,NaN,2021-11-08T22:22:11+01:00,https://opendata.paris.fr/api/v2/catalog/datas...,NaN,NaN,NaN
2172,13323,https://www.paris.fr/evenements/lonely-band-re...,Lonely Band : release party,+ Astrønne + Madone,<p>Chanter et écrire en français a permis à Lo...,2021-12-11 20:00:38,2021-12-11 23:00:38,2021-12-11T20:00:38+01:00_2021-12-11T23:00:38+...,Le samedi 11 décembre 2021<br />de 20h00 à 23h...,https://cdn.paris.fr/qfapv4/2021/11/08/medium-...,...,payant,NaN,conseillée,https://www.lehasardludique.paris/concert/2021...,NaN,2021-11-08T